# JUICE RPWI HF SID20 (Burst rich): L1a QL -- 2024/1/16

# Import lib

In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors

# CDF and Directory setting: set by User

In [ ]:
# The latest CDF library https://spdf.gsfc.nasa.gov/pub/software/cdf/dist/latest/
from spacepy import pycdf

import os
os.environ["CDF_LIB"] = "/Applications/cdf/cdf39_0-dist/lib"

# *** Library ***
sys.path.append('./lib/')
import juice_cdf_lib as juice_cdf
import juice_sid20_lib as juice_sid20
import juice_math_lib as juice_math

# Setting and Read CDF file: set by User

In [ ]:
# *** Mode set ***
band_mode = 0                           # 0: sum    1: /Hz
unit_mode = 0                           # 0: raw    1: dBm＠ADC  2: V@HF   3:V2@HF   4:V2@RWI
f_mode = 0                              # 0: linear   1:log  in frequency
dump_mode = 1                           # 0: no-dump  1:plot dump

# *** Parameter set ***
cal = 0                                 # 0: background     1: CAL
p_raw_max = 8.5                         # background: 7.5   CAL: 10
p_raw_min = 1.0                         # background: 2.5   CAL: 5

# *** Conversion factor: cal_mode ***
cf, p_max, p_min = juice_cdf.cal_factors(unit_mode, p_raw_max, p_raw_min)
print("conversion factor:", cf, "    MAX-min:", p_max, p_min)

In [ ]:
# *** Directory set: set by User ***
work_dir = '/Users/user/0-python/JUICE_python/ql/'   # Plot dump folder

# *** Ver.2 ***
# 202310 -- SAMPLE
data_dir = '/Users/user/0-python/JUICE_data/test-CCSDS/sample/cdf/'        # CDF data folder
data_name = 'SID20_20231024-0042.cdf'

"""
# 202310 -- FS
data_dir = '/Users/user/0-python/JUICE_data/test-TMIDX/202311_FS/cdf/'        # CDF data folder
data_name = 'SID20_Seq01.cdf'
data_name = 'SID20_Seq02.cdf'
data_name = 'SID20_Seq16.cdf'
data_name = 'SID20_Seq18.cdf'
data_name = 'SID20_Seq19.cdf'
data_name = 'SID20_Seq20.cdf'

# 202312 -- Checkout
data_dir = '/Users/user/0-python/JUICE_data/test-TMIDX/202312_C/cdf/'    # 202312_FS test folder
data_name = 'SID20_RPWI_NEWv2_SCI.cdf'
"""

# get CDF data

In [ ]:
cdf_file = data_dir + data_name
%ls $cdf_file

cdf = pycdf.CDF(cdf_file)
data = juice_sid20.juice_getdata_hf_sid20(cdf, cf)

date1 = data.epoch[0]
date1 = date1.strftime('%Y/%m/%d %R:%S')
date2 = data.epoch[-1]
date2 = date2.strftime('%Y/%m/%d %R:%S')
str_date = date1 + "  -  " + date2
print(str_date)

n_freq = data.EuEu.shape[1]
n_time = data.EuEu.shape[0]
n_time0 = n_time-1
print("data size:", data.EuEu.shape, n_time, n_freq)

In [ ]:
# Mode 
N_ch = data.U_selected[0] + data.V_selected[0] + data.W_selected[0]
if (data.complex[0] == 0):
    print("complex:", data.complex[0], " (power 3-ch) <Table 6.3.2>")
    N_component = 3
elif (data.complex[0] == 1):
    print("complex:", data.complex[0], " (spectral matrix 3-ch) <Table 6.3.3>")
    N_component = 9
else:
    print("complex:", data.complex[0], " (3D spectral matrix 3-ch) <Table 6.3.4>")
    N_component = 21
print("Ch:", N_ch, "  (U:", data.U_selected[0], "  V:", data.V_selected[0], "  W:", data.W_selected[0], ")")
N_frequency = data.B0_step[0] * data.B0_subdiv[0]
print("==> Component", N_component, "   Num-frequency", N_frequency, "  Num-sweep:", n_time)

# Data for checks

In [ ]:
from datetime import timedelta

# Time resoution: 1/n_block [sec]
Epoch_1d = data.epoch.tolist()
Epoch_2d = []
for i in range(data.N_block[0]):
    Epoch_2d += Epoch_1d
for i in range(n_time):
    for j in range(data.N_block[0]):
        Epoch_2d[i*data.N_block[0] + j] = Epoch_1d[i] + timedelta(seconds = j/data.N_block[0])

# Frequency
freq_1d       = data.frequency[0].tolist()
freq1         = freq_1d[0:N_frequency]
freq_width_1d = data.freq_width[0].tolist()
freq_width1   = freq_width_1d[0:N_frequency]

# Data
EuEu_2d    = data.EuEu.reshape(n_time * data.N_block[0], N_frequency);     EuEu_2d    = EuEu_2d.transpose()
EvEv_2d    = data.EvEv.reshape(n_time * data.N_block[0], N_frequency);     EvEv_2d    = EvEv_2d.transpose()
EwEw_2d    = data.EwEw.reshape(n_time * data.N_block[0], N_frequency);     EwEw_2d    = EwEw_2d.transpose()
E_DoLuv_2d = data.E_DoLuv.reshape(n_time * data.N_block[0], N_frequency);  E_DoLuv_2d = E_DoLuv_2d.transpose()
E_DoLvw_2d = data.E_DoLvw.reshape(n_time * data.N_block[0], N_frequency);  E_DoLvw_2d = E_DoLvw_2d.transpose()
E_DoLwu_2d = data.E_DoLwu.reshape(n_time * data.N_block[0], N_frequency);  E_DoLwu_2d = E_DoLwu_2d.transpose()
E_DoCuv_2d = data.E_DoCuv.reshape(n_time * data.N_block[0], N_frequency);  E_DoCuv_2d = E_DoCuv_2d.transpose()
E_DoCvw_2d = data.E_DoCvw.reshape(n_time * data.N_block[0], N_frequency);  E_DoCvw_2d = E_DoCvw_2d.transpose()
E_DoCwu_2d = data.E_DoCwu.reshape(n_time * data.N_block[0], N_frequency);  E_DoCwu_2d = E_DoCwu_2d.transpose()
E_ANGuv_2d = data.E_ANGuv.reshape(n_time * data.N_block[0], N_frequency);  E_ANGuv_2d = E_ANGuv_2d.transpose()
E_ANGvw_2d = data.E_ANGvw.reshape(n_time * data.N_block[0], N_frequency);  E_ANGvw_2d = E_ANGvw_2d.transpose()
E_ANGwu_2d = data.E_ANGwu.reshape(n_time * data.N_block[0], N_frequency);  E_ANGwu_2d = E_ANGwu_2d.transpose()
freq_width_2d = data.freq_width.reshape(n_time * data.N_block[0], N_frequency); freq_width_2d = freq_width_2d.transpose()
if band_mode == 1:
    EuEu_2d = EuEu_2d / freq_width_2d / 1000
    EvEv_2d = EvEv_2d / freq_width_2d / 1000
    EwEw_2d = EwEw_2d / freq_width_2d / 1000
    """
    print("frequency width:", freq_width1)
    """
#
f_min = freq_1d[0]
f_max = freq_1d[N_frequency-1]
power_str = juice_cdf.power_label(band_mode, unit_mode)
print(power_str, "   f_MAX/min:", f_max, f_min, "   num of f, step, window:", N_frequency, data.freq_step[0][0], data.freq_width[0][0])

## Raw

In [ ]:
fig = plt.figure(figsize=(12, 14))
ax1 = fig.add_subplot(8, 1, 1)
ax2 = fig.add_subplot(8, 1, 2)
ax3 = fig.add_subplot(8, 1, 3)
ax4 = fig.add_subplot(8, 1, 4)
ax5 = fig.add_subplot(8, 1, 5)
ax6 = fig.add_subplot(8, 1, 6)
ax7 = fig.add_subplot(8, 1, 7)
ax8 = fig.add_subplot(8, 1, 8)

ax1.plot(np.ravel(data.EuEu), '-r', linewidth=1.2, label='uu')
ax1.plot(np.ravel(data.EvEv), '-g', linewidth=0.8, label='vv')
ax1.plot(np.ravel(data.EwEw), '-b', linewidth=0.5, label='ww')
ax1.set_yscale('log')
#
if data.complex[0] > 0:    # Matrix
    ax2.plot(np.ravel(data.EuEv_re), '-r', linewidth=1.0, label='uv_re')
    ax2.plot(np.ravel(data.EvEw_re), '-g', linewidth=0.8, label='vw_re')
    ax2.plot(np.ravel(data.EwEu_re), '-b', linewidth=0.6, label='wu_re')
    ax2.plot(np.ravel(data.EuEv_im), ':r', linewidth=1.2,  label='uv_im')
    ax2.plot(np.ravel(data.EvEw_im), ':g', linewidth=1.0,  label='vw_im')
    ax2.plot(np.ravel(data.EwEu_im), ':b', linewidth=0.8,  label='wu_im')
    #
    ax3.plot(np.ravel(data.E_DoPuv), '-r', linewidth=1.0, label='uv')
    ax3.plot(np.ravel(data.E_DoPvw), '-g', linewidth=0.8, label='vw')
    ax3.plot(np.ravel(data.E_DoPwu), '-b', linewidth=0.6, label='wu')
    #
    ax4.plot(np.ravel(data.E_DoLuv), '-r', linewidth=1.0, label='uv')
    ax4.plot(np.ravel(data.E_DoLvw), '-g', linewidth=0.8, label='vw')
    ax4.plot(np.ravel(data.E_DoLwu), '-b', linewidth=0.6, label='wu')
    #
    ax5.plot(np.ravel(data.E_DoCuv), '-r', linewidth=1.0, label='uv')
    ax5.plot(np.ravel(data.E_DoCvw), '-g', linewidth=0.8, label='vw')
    ax5.plot(np.ravel(data.E_DoCwu), '-b', linewidth=0.6, label='wu')
    #
    ax6.plot(np.ravel(data.E_ANGuv), '-r', linewidth=1.0, label='uv')
    ax6.plot(np.ravel(data.E_ANGvw), '-g', linewidth=0.8, label='vw')
    ax6.plot(np.ravel(data.E_ANGwu), '-b', linewidth=0.6, label='wu')

ax7.plot(np.ravel(data.frequency),     '-r', linewidth=0.6, label='frequency')
ax7.plot(np.ravel(data.freq_step*10),  '-g', linewidth=0.8, label='step*10')
ax7.plot(np.ravel(data.freq_width*10), ':b', linewidth=1.0, label='width*10')
ax8.plot(np.ravel(data.epoch[:]), '.')

ax1.set_ylabel(power_str)
if data.complex[0] > 0:    # Matrix
    ax2.set_ylabel('Real & Img')
    ax3.set_ylabel('Deg. Pol.')
    ax4.set_ylabel('Deg. Linear Pol.')
    ax5.set_ylabel('Deg. Circular Pol.')
    ax6.set_ylabel('Linear Deg.')
ax7.set_ylabel('frequency [kHz]')
ax8.set_ylabel('Date-Time')
ax8.set_xlabel(str_date)
#
title_label = '[JUICE/RPWI HF Burst-Rich (SID-20)]  ' + data_name
ax1.set_title(title_label)
ax1.legend(loc='upper right', fontsize=8); ax2.legend(loc='upper right', fontsize=8); ax3.legend(loc='upper right', fontsize=8); 
ax4.legend(loc='upper right', fontsize=8); ax5.legend(loc='upper right', fontsize=8); ax6.legend(loc='upper right', fontsize=8); 
ax7.legend(loc='upper right', fontsize=8)

# range: X-axis
"""
xlim=[0, N_frequency*1-1]
# xlim=[50, 60]
ax1.set_xlim(xlim); ax2.set_xlim(xlim); ax3.set_xlim(xlim); ax4.set_xlim(xlim); ax5.set_xlim(xlim); ax6.set_xlim(xlim); ax7.set_xlim(xlim)
# range: Y-axis
ylim=[10**p_raw_min, 10**p_raw_max]
ax1.set_ylim(ylim)
"""
if data.complex[0] > 0:    # Matrix
    # ylim=[-10**p_raw_max, 10**p_raw_max]; ax2.set_ylim(ylim)
    ylim=[-0.05, 1.05]; ax3.set_ylim(ylim); ax4.set_ylim(ylim)
    ylim=[-1.05, 1.05]; ax5.set_ylim(ylim)
    ylim=[-0.05, 185.]; ax6.set_ylim(ylim)
ylim=[f_min, f_max]
ax7.set_ylim(ylim)

# Plot
fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_raw.png'
    fig.savefig(png_fname)

## Spec

In [ ]:
# Sweep_num
n0 = 0
n1 = np.int16(n_time0/2)
n2 = n_time0  # *5//7
# X-range
f_min0 = f_min;  f_max0 = f_max
# Y-range
p_min0 = p_min;  p_max0 = p_max

fig = plt.figure(figsize=(12, 12))
ax1 = fig.add_subplot(4, 1, 1);   ax5 = fig.add_subplot(4, 1, 2);  ax6 = fig.add_subplot(4, 1, 3);  ax7 = fig.add_subplot(4, 1, 4)
# ax4 = fig.add_subplot(6, 1, 5); ax3 = fig.add_subplot(6, 1, 6)

ax1.plot(freq1, data.EuEu[n0][0:N_frequency], ':r', linewidth=1.0, label='uu init')
ax1.plot(freq1, data.EvEv[n0][0:N_frequency], ':g', linewidth=0.8, label='vv init')
ax1.plot(freq1, data.EwEw[n0][0:N_frequency], ':b', linewidth=0.6, label='ww init')
ax1.plot(freq1, data.EuEu[n1][0:N_frequency], '-.r', linewidth=1.0, label='uu half')
ax1.plot(freq1, data.EvEv[n1][0:N_frequency], '-.g', linewidth=0.8, label='vv half')
ax1.plot(freq1, data.EwEw[n1][0:N_frequency], '-.b', linewidth=0.6, label='ww half')
ax1.plot(freq1, data.EuEu[n2][0:N_frequency], '-r', linewidth=1.0, label='uu end')
ax1.plot(freq1, data.EvEv[n2][0:N_frequency], '-g', linewidth=0.8, label='vv end')
ax1.plot(freq1, data.EwEw[n2][0:N_frequency], '-b', linewidth=0.6, label='ww end')
ax1.set_yscale('log')
if f_mode == 1:
    ax1.set_xscale('log')
#
if data.complex[0] > 0:    # Matrix
    """
    ax3.plot(freq1, data.EuEv_re[n1][0:N_frequency], '-r', linewidth=1.0, label='uv_re half')
    ax3.plot(freq1, data.EvEw_re[n1][0:N_frequency], '-g', linewidth=0.8, label='vw_re half')
    ax3.plot(freq1, data.EwEu_re[n1][0:N_frequency], '-b', linewidth=0.6, label='wu_re half')
    ax3.plot(freq1, data.EuEv_im[n1][0:N_frequency], ':r', linewidth=1.2,  label='uv_im half')
    ax3.plot(freq1, data.EvEw_im[n1][0:N_frequency], ':g', linewidth=1.0,  label='vw_im half')
    ax3.plot(freq1, data.EwEu_im[n1][0:N_frequency], ':b', linewidth=0.8,  label='wu_im half')
    #
    ax4.plot(freq1, data.E_DoPuv[n0][0:N_frequency], ':r', linewidth=1.0, label='uv init')
    ax4.plot(freq1, data.E_DoPvw[n0][0:N_frequency], ':g', linewidth=0.8, label='vw init')
    ax4.plot(freq1, data.E_DoPwu[n0][0:N_frequency], ':b', linewidth=0.6, label='wu init')
    ax4.plot(freq1, data.E_DoPuv[n1][0:N_frequency], '-.r', linewidth=1.2,  label='uv half')
    ax4.plot(freq1, data.E_DoPvw[n1][0:N_frequency], '-.g', linewidth=1.0,  label='vw half')
    ax4.plot(freq1, data.E_DoPwu[n1][0:N_frequency], '-.b', linewidth=0.8,  label='wu half')
    ax4.plot(freq1, data.E_DoPuv[n2][0:N_frequency], '-r', linewidth=1.2,  label='uv end')
    ax4.plot(freq1, data.E_DoPvw[n2][0:N_frequency], '-g', linewidth=1.0,  label='vw end')
    ax4.plot(freq1, data.E_DoPwu[n2][0:N_frequency], '-b', linewidth=0.8,  label='wu end')
    #
    """
    ax5.plot(freq1, data.E_DoLuv[n0][0:N_frequency], ':r', linewidth=1.0, label='uv init')
    ax5.plot(freq1, data.E_DoLvw[n0][0:N_frequency], ':g', linewidth=0.8, label='vw init')
    ax5.plot(freq1, data.E_DoLwu[n0][0:N_frequency], ':b', linewidth=0.6, label='wu init')
    ax5.plot(freq1, data.E_DoLuv[n1][0:N_frequency], '-.r', linewidth=1.2,  label='uv half')
    ax5.plot(freq1, data.E_DoLvw[n1][0:N_frequency], '-.g', linewidth=1.0,  label='vw half')
    ax5.plot(freq1, data.E_DoLwu[n1][0:N_frequency], '-.b', linewidth=0.8,  label='wu half')
    ax5.plot(freq1, data.E_DoLuv[n2][0:N_frequency], '-r', linewidth=1.2,  label='uv end')
    ax5.plot(freq1, data.E_DoLvw[n2][0:N_frequency], '-g', linewidth=1.0,  label='vw end')
    ax5.plot(freq1, data.E_DoLwu[n2][0:N_frequency], '-b', linewidth=0.8,  label='wu end')
    #
    ax6.plot(freq1, data.E_DoCuv[n0][0:N_frequency], ':r', linewidth=1.0, label='uv init')
    ax6.plot(freq1, data.E_DoCvw[n0][0:N_frequency], ':g', linewidth=0.8, label='vw init')
    ax6.plot(freq1, data.E_DoCwu[n0][0:N_frequency], ':b', linewidth=0.6, label='wu init')
    ax6.plot(freq1, data.E_DoCuv[n1][0:N_frequency], '-.r', linewidth=1.2,  label='uv half')
    ax6.plot(freq1, data.E_DoCvw[n1][0:N_frequency], '-.g', linewidth=1.0,  label='vw half')
    ax6.plot(freq1, data.E_DoCwu[n1][0:N_frequency], '-.b', linewidth=0.8,  label='wu half')
    ax6.plot(freq1, data.E_DoCuv[n2][0:N_frequency], '-r', linewidth=1.2,  label='uv end')
    ax6.plot(freq1, data.E_DoCvw[n2][0:N_frequency], '-g', linewidth=1.0,  label='vw end')
    ax6.plot(freq1, data.E_DoCwu[n2][0:N_frequency], '-b', linewidth=0.8,  label='wu end')
    #
    ax7.plot(freq1, data.E_ANGuv[n0][0:N_frequency], ':r', linewidth=1.0, label='uv init')
    ax7.plot(freq1, data.E_ANGvw[n0][0:N_frequency], ':g', linewidth=0.8, label='vw init')
    ax7.plot(freq1, data.E_ANGwu[n0][0:N_frequency], ':b', linewidth=0.6, label='wu init')
    ax7.plot(freq1, data.E_ANGuv[n1][0:N_frequency], '-.r', linewidth=1.2,  label='uv half')
    ax7.plot(freq1, data.E_ANGvw[n1][0:N_frequency], '-.g', linewidth=1.0,  label='vw half')
    ax7.plot(freq1, data.E_ANGwu[n1][0:N_frequency], '-.b', linewidth=0.8,  label='wu half')
    ax7.plot(freq1, data.E_ANGuv[n2][0:N_frequency], '-r', linewidth=1.2,  label='uv end')
    ax7.plot(freq1, data.E_ANGvw[n2][0:N_frequency], '-g', linewidth=1.0,  label='vw end')
    ax7.plot(freq1, data.E_ANGwu[n2][0:N_frequency], '-b', linewidth=0.8,  label='wu end')
    #
    if f_mode == 1:
        ax5.set_xscale('log');  ax6.set_xscale('log');  ax7.set_xscale('log')   # ax3.set_xscale('log');  ax4.set_xscale('log')

# Label
ax1.set_ylabel(power_str)
if data.complex[0] > 0:    # Matrix
    ax5.set_ylabel('Deg. Linear Pol.')
    ax6.set_ylabel('Deg. Circular Pol.')
    ax7.set_ylabel('Linear Deg.')
    # ax4.set_ylabel('Deg. Pol.')
    # ax3.set_ylabel('Real & Img')
ax7.set_xlabel('frequency [Hz]')
#
date1 = data.epoch[n0];  date1 = date1.strftime('%Y/%m/%d %R:%S')
date2 = data.epoch[n1];  date2 = date2.strftime('%Y/%m/%d %R:%S')
date3 = data.epoch[n2];  date3 = date3.strftime('%Y/%m/%d %R:%S')
title_date = "[" + data_name + "]  " + date1 + "  -  " + date2 + "  -  " + date3 
ax1.set_title(title_date)
ax1.legend(loc='upper right', fontsize=8)
ax5.legend(loc='upper right', fontsize=8); ax6.legend(loc='upper right', fontsize=8);  ax7.legend(loc='upper right', fontsize=8)
# ax3.legend(loc='upper right', fontsize=8); ax4.legend(loc='upper right', fontsize=8); 

# range: X-axis
xlim=[f_min0, f_max0]
ax1.set_xlim(xlim); ax5.set_xlim(xlim); ax6.set_xlim(xlim); ax7.set_xlim(xlim); # ax3.set_xlim(xlim); ax4.set_xlim(xlim); 
# range: Y-axis
# ylim=[10**p_min0, 10**p_max0]; ax1.set_ylim(ylim)
if data.complex[0] > 0:    # Matrix
    ylim=[-0.05, 1.05]; ax5.set_ylim(ylim);  # ax4.set_ylim(ylim); 
    ylim=[-1.05, 1.05]; ax6.set_ylim(ylim)
    ylim=[-0.05, 185.]; ax7.set_ylim(ylim)
    # ylim=[-10**p_raw_max, 10**p_raw_max]; ax3.set_ylim(ylim)

# Plot
fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_spec.png'
    if f_mode == 1:
        png_fname = work_dir+data_name+'_spec-log.png'
    fig.savefig(png_fname)

# Peak
print("[First]")
peak_E = data.EuEu[n0][0:N_frequency]; peak_f = np.argmax(peak_E); print("Peak EuEu:", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(freq1[peak_f]), "kHz")
peak_E = data.EvEv[n0][0:N_frequency]; peak_f = np.argmax(peak_E); print("Peak EvEv:", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(freq1[peak_f]), "kHz")
peak_E = data.EwEw[n0][0:N_frequency]; peak_f = np.argmax(peak_E); print("Peak EwEw:", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(freq1[peak_f]), "kHz")
print("[Mid]")
peak_E = data.EuEu[n1][0:N_frequency]; peak_f = np.argmax(peak_E); print("Peak EuEu:", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(freq1[peak_f]), "kHz")
peak_E = data.EvEv[n1][0:N_frequency]; peak_f = np.argmax(peak_E); print("Peak EvEv:", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(freq1[peak_f]), "kHz")
peak_E = data.EwEw[n1][0:N_frequency]; peak_f = np.argmax(peak_E); print("Peak EwEw:", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(freq1[peak_f]), "kHz")
print("[Last]")
peak_E = data.EuEu[n2][0:N_frequency]; peak_f = np.argmax(peak_E); print("Peak EuEu:", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(freq1[peak_f]), "kHz")
peak_E = data.EvEv[n2][0:N_frequency]; peak_f = np.argmax(peak_E); print("Peak EvEv:", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(freq1[peak_f]), "kHz")
peak_E = data.EwEw[n2][0:N_frequency]; peak_f = np.argmax(peak_E); print("Peak EwEw:", '{:.2e}'.format(peak_E[peak_f]), "at", '{:.1f}'.format(freq1[peak_f]), "kHz")

## FT

In [ ]:
# Y-range
f_min0 = f_min;      f_max0 = f_max
# Z-range
p_min0 = p_min;      p_max0 = p_max

fig2d = plt.figure(figsize=[12,14])
ax1 = fig2d.add_subplot(3, 1, 1)
ax2 = fig2d.add_subplot(3, 1, 2)
ax3 = fig2d.add_subplot(3, 1, 3)

# Y-axis
ax1.set_ylim(f_min0, f_max0);      ax2.set_ylim(f_min0, f_max0);      ax3.set_ylim(f_min0, f_max0)
if f_mode == 1:
    ax1.set_yscale('log');         ax2.set_yscale('log');             ax3.set_yscale('log')
ax1.set_ylabel('Frequency [kHz]'); ax2.set_ylabel('Frequency [kHz]'); ax3.set_ylabel('Frequency [kHz]')

# X-axis
ax1.set_title("[" + data_name + "]  " + 'EuEu');             ax2.set_title('EvEv');             ax3.set_title('EwEw')
ax3.set_xlabel(str_date)

# Plot
p1 = ax1.pcolormesh(Epoch_2d, freq1, EuEu_2d, norm=colors.LogNorm(vmin=10**p_min0, vmax=10**p_max0), cmap='jet')
p2 = ax2.pcolormesh(Epoch_2d, freq1, EvEv_2d, norm=colors.LogNorm(vmin=10**p_min0, vmax=10**p_max0), cmap='jet')
p3 = ax3.pcolormesh(Epoch_2d, freq1, EwEw_2d, norm=colors.LogNorm(vmin=10**p_min0, vmax=10**p_max0), cmap='jet')

pp1 = fig2d.colorbar(p1, ax=ax1, orientation="vertical");  pp1.set_label(power_str)
pp2 = fig2d.colorbar(p2, ax=ax2, orientation="vertical");  pp2.set_label(power_str)
pp3 = fig2d.colorbar(p3, ax=ax3, orientation="vertical");  pp3.set_label(power_str)

plt.show()
if dump_mode == 1:
    png_fname = work_dir+data_name+'_FT.png'
    if f_mode == 1:
        png_fname = work_dir+data_name+'_FT-log.png'
    fig2d.savefig(png_fname)

## Ang-T

In [ ]:
# Y-range
f_min0 = f_min;      f_max0 = f_max
# Z-range
p_min0 = p_min;      p_max0 = p_max

fig2d = plt.figure(figsize=[12,30])
ax1 = fig2d.add_subplot(9, 1, 1);  ax2 = fig2d.add_subplot(9, 1, 2);  ax3 = fig2d.add_subplot(9, 1, 3)
ax4 = fig2d.add_subplot(9, 1, 4);  ax5 = fig2d.add_subplot(9, 1, 5);  ax6 = fig2d.add_subplot(9, 1, 6)
ax7 = fig2d.add_subplot(9, 1, 7);  ax8 = fig2d.add_subplot(9, 1, 8);  ax9 = fig2d.add_subplot(9, 1, 9)

# Y-axis
ax1.set_ylim(f_min0, f_max0);      ax2.set_ylim(f_min0, f_max0);      ax3.set_ylim(f_min0, f_max0)
ax4.set_ylim(f_min0, f_max0);      ax5.set_ylim(f_min0, f_max0);      ax6.set_ylim(f_min0, f_max0)
ax7.set_ylim(f_min0, f_max0);      ax8.set_ylim(f_min0, f_max0);      ax9.set_ylim(f_min0, f_max0)
if f_mode == 1:
    ax1.set_yscale('log');         ax2.set_yscale('log');             ax3.set_yscale('log')
    ax4.set_yscale('log');         ax5.set_yscale('log');             ax6.set_yscale('log')
    ax7.set_yscale('log');         ax8.set_yscale('log');             ax9.set_yscale('log')
ax1.set_ylabel('Frequency [kHz]'); ax2.set_ylabel('Frequency [kHz]'); ax3.set_ylabel('Frequency [kHz]')
ax4.set_ylabel('Frequency [kHz]'); ax5.set_ylabel('Frequency [kHz]'); ax6.set_ylabel('Frequency [kHz]')
ax7.set_ylabel('Frequency [kHz]'); ax8.set_ylabel('Frequency [kHz]'); ax9.set_ylabel('Frequency [kHz]')

# X-axis
ax1.set_title("[" + data_name + "]  " + 'Deg. Lin. uv');     ax2.set_title('Deg. Lin. vw');     ax3.set_title('Deg. Lin. wu')
ax4.set_title('Deg. Cir. uv');     ax5.set_title('Deg. Cir. vw');     ax6.set_title('Deg. Cir. wu')
ax7.set_title('Ang. uv');          ax8.set_title('Ang. vw');          ax9.set_title('Ang. wu')
ax9.set_xlabel(str_date)

# Plot
p1 = ax1.pcolormesh(Epoch_2d, freq1, E_DoLuv_2d, norm=colors.Normalize(vmin=-0.01, vmax=1.01), cmap='Oranges')
p2 = ax2.pcolormesh(Epoch_2d, freq1, E_DoLvw_2d, norm=colors.Normalize(vmin=-0.01, vmax=1.01), cmap='Oranges')
p3 = ax3.pcolormesh(Epoch_2d, freq1, E_DoLwu_2d, norm=colors.Normalize(vmin=-0.01, vmax=1.01), cmap='Oranges')
p4 = ax4.pcolormesh(Epoch_2d, freq1, E_DoCuv_2d, norm=colors.Normalize(vmin=-1.01, vmax=1.01), cmap='bwr')
p5 = ax5.pcolormesh(Epoch_2d, freq1, E_DoCvw_2d, norm=colors.Normalize(vmin=-1.01, vmax=1.01), cmap='bwr')
p6 = ax6.pcolormesh(Epoch_2d, freq1, E_DoCwu_2d, norm=colors.Normalize(vmin=-1.01, vmax=1.01), cmap='bwr')
p7 = ax7.pcolormesh(Epoch_2d, freq1, E_ANGuv_2d, norm=colors.Normalize(vmin=-1.00, vmax=181.), cmap='bwr')
p8 = ax8.pcolormesh(Epoch_2d, freq1, E_ANGvw_2d, norm=colors.Normalize(vmin=-1.00, vmax=181.), cmap='bwr')
p9 = ax9.pcolormesh(Epoch_2d, freq1, E_ANGwu_2d, norm=colors.Normalize(vmin=-1.00, vmax=181.), cmap='bwr')

pp1 = fig2d.colorbar(p1, ax=ax1, orientation="vertical")
pp2 = fig2d.colorbar(p2, ax=ax2, orientation="vertical")
pp3 = fig2d.colorbar(p3, ax=ax3, orientation="vertical")
pp4 = fig2d.colorbar(p4, ax=ax4, orientation="vertical")
pp5 = fig2d.colorbar(p5, ax=ax5, orientation="vertical")
pp6 = fig2d.colorbar(p6, ax=ax6, orientation="vertical")
pp7 = fig2d.colorbar(p7, ax=ax7, orientation="vertical")
pp8 = fig2d.colorbar(p8, ax=ax8, orientation="vertical")
pp9 = fig2d.colorbar(p9, ax=ax9, orientation="vertical")

plt.show()
if dump_mode == 1:
    png_fname = work_dir+data_name+'_AngT.png'
    if f_mode == 1:
        png_fname = work_dir+data_name+'_AngT-log.png'
    fig2d.savefig(png_fname)


# CLEANED data

In [ ]:
EuEu_c = juice_math.clean_rfi(np.ravel(data.EuEu))
EvEv_c = juice_math.clean_rfi(np.ravel(data.EvEv))
EwEw_c = juice_math.clean_rfi(np.ravel(data.EwEw))
EuEu_c = EuEu_c.reshape(n_time, n_freq)
EvEv_c = EvEv_c.reshape(n_time, n_freq)
EwEw_c = EwEw_c.reshape(n_time, n_freq)
EuEu_c_2d = EuEu_c.transpose()
EvEv_c_2d = EvEv_c.transpose()
EwEw_c_2d = EwEw_c.transpose()

## CLEANED - Spec

In [ ]:
# Sweep_num
n0 = 0
n1 = np.int16(n_time0/2)
n2 = n_time0
# X-range
f_min0 = f_min;  f_max0 = f_max
# Y-range
p_min0 = p_min;  p_max0 = p_max

fig = plt.figure(figsize=(12, 8))
ax1 = fig.add_subplot(2, 1, 1)
ax2 = fig.add_subplot(2, 1, 2)

ax1.plot(freq1, data.EuEu[n0][0:N_frequency], ':r', linewidth=1.0, label='uu init')
ax1.plot(freq1, data.EvEv[n0][0:N_frequency], ':g', linewidth=0.8, label='vv init')
ax1.plot(freq1, data.EwEw[n0][0:N_frequency], ':b', linewidth=0.6, label='ww init')
ax1.plot(freq1, data.EuEu[n1][0:N_frequency], '-.r', linewidth=1.0, label='uu half')
ax1.plot(freq1, data.EvEv[n1][0:N_frequency], '-.g', linewidth=0.8, label='vv half')
ax1.plot(freq1, data.EwEw[n1][0:N_frequency], '-.b', linewidth=0.6, label='ww half')
ax1.plot(freq1, data.EuEu[n2][0:N_frequency], '-r', linewidth=1.0, label='uu end')
ax1.plot(freq1, data.EvEv[n2][0:N_frequency], '-g', linewidth=0.8, label='vv end')
ax1.plot(freq1, data.EwEw[n2][0:N_frequency], '-b', linewidth=0.6, label='ww end')
ax2.plot(freq1, EuEu_c[n0][0:N_frequency], ':r', linewidth=1.0, label='uu init cleaned')
ax2.plot(freq1, EvEv_c[n0][0:N_frequency], ':g', linewidth=0.8, label='vv init cleaned')
ax2.plot(freq1, EwEw_c[n0][0:N_frequency], ':b', linewidth=0.6, label='ww init cleaned')
ax2.plot(freq1, EuEu_c[n1][0:N_frequency], '-.r', linewidth=1.0, label='uu half cleaned')
ax2.plot(freq1, EvEv_c[n1][0:N_frequency], '-.g', linewidth=0.8, label='vv half cleaned')
ax2.plot(freq1, EwEw_c[n1][0:N_frequency], '-.b', linewidth=0.6, label='ww half cleaned')
ax2.plot(freq1, EuEu_c[n2][0:N_frequency], '-r', linewidth=1.0, label='uu end cleaned')
ax2.plot(freq1, EvEv_c[n2][0:N_frequency], '-g', linewidth=0.8, label='vv end cleaned')
ax2.plot(freq1, EwEw_c[n2][0:N_frequency], '-b', linewidth=0.6, label='ww end cleaned')
ax1.set_yscale('log');      ax2.set_yscale('log')
if f_mode == 1:
    ax1.set_xscale('log');  ax2.set_xscale('log')

# Label
ax1.set_ylabel(power_str);  ax2.set_ylabel(power_str)
ax2.set_xlabel('frequency [Hz]')
#
date1 = data.epoch[n0];  date1 = date1.strftime('%Y/%m/%d %R:%S')
date2 = data.epoch[n1];  date2 = date2.strftime('%Y/%m/%d %R:%S')
date3 = data.epoch[n2];  date3 = date3.strftime('%Y/%m/%d %R:%S')
title_date = date1 + "  -  " + date2 + "  -  " + date3 
ax1.set_title("[" + data_name + "]  " + title_date)
ax1.legend(loc='upper right', fontsize=8); ax2.legend(loc='upper right', fontsize=8)

# range: X-axis
xlim=[f_min0, f_max0]
ax1.set_xlim(xlim); ax2.set_xlim(xlim);

# Plot
fig.show
if dump_mode == 1:
    png_fname = work_dir+data_name+'_spec-c.png'
    if f_mode == 1:
        png_fname = work_dir+data_name+'_spec-log-c.png'
    # fig2d.savefig(png_fname)

## CLEANED -- FT

In [ ]:
# Y-range
f_min0 = f_min;      f_max0 = f_max
# Z-range
p_min0 = p_min;      p_max0 = p_max

fig2d = plt.figure(figsize=[12,14])
ax1 = fig2d.add_subplot(3, 1, 1)
ax2 = fig2d.add_subplot(3, 1, 2)
ax3 = fig2d.add_subplot(3, 1, 3)

# Y-axis
ax1.set_ylim(f_min0, f_max0);      ax2.set_ylim(f_min0, f_max0);      ax3.set_ylim(f_min0, f_max0)
if f_mode == 1:
    ax1.set_yscale('log');         ax2.set_yscale('log');             ax3.set_yscale('log')
ax1.set_ylabel('Frequency [kHz]'); ax2.set_ylabel('Frequency [kHz]'); ax3.set_ylabel('Frequency [kHz]')

# X-axis
ax1.set_title("[" + data_name + "]  " + 'EuEu cleaned');     ax2.set_title('EvEv cleaned');     ax3.set_title('EwEw cleaned')
ax3.set_xlabel(str_date)

# Plot
p1 = ax1.pcolormesh(Epoch_1d, freq_1d, EuEu_c_2d, norm=colors.LogNorm(vmin=10**p_min0, vmax=10**p_max0), cmap='jet')
p2 = ax2.pcolormesh(Epoch_1d, freq_1d, EvEv_c_2d, norm=colors.LogNorm(vmin=10**p_min0, vmax=10**p_max0), cmap='jet')
p3 = ax3.pcolormesh(Epoch_1d, freq_1d, EwEw_c_2d, norm=colors.LogNorm(vmin=10**p_min0, vmax=10**p_max0), cmap='jet')

pp1 = fig2d.colorbar(p1, ax=ax1, orientation="vertical");  pp1.set_label(power_str)
pp2 = fig2d.colorbar(p2, ax=ax2, orientation="vertical");  pp2.set_label(power_str)
pp3 = fig2d.colorbar(p3, ax=ax3, orientation="vertical");  pp3.set_label(power_str)

plt.show()
if dump_mode == 1:
    png_fname = work_dir+data_name+'_FT-c.png'
    if f_mode == 1:
        png_fname = work_dir+data_name+'_FT-log-c.png'
    # fig2d.savefig(png_fname)